In [ ]:
"""
MASTER EXECUTION SCRIPT
======================
Complete analysis pipeline from data loading to insights generation

This script runs the entire analysis workflow:
1. Load and clean data
2. Perform exploratory analysis
3. Extract deep insights
4. Generate visualizations
5. Create comprehensive report

Author: Abhinav Anand
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🚀 TRADER BEHAVIOR ANALYSIS - COMPLETE PIPELINE")
print("="*80)
print(f"\nExecution started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# ============================================================================
# STEP 1: DATA LOADING
# ============================================================================

print("\n" + "="*80)
print("STEP 1: LOADING DATA")
print("="*80)

def load_and_prepare_data():
    """Load and prepare both datasets"""

    print("\n📂 Loading datasets...")

    # Load Fear & Greed Index
    try:
        fear_greed = pd.read_csv('fear_greed_index.csv')
        print(f"✅ Fear & Greed Index loaded: {fear_greed.shape}")
    except Exception as e:
        print(f"❌ Error loading fear_greed_index.csv: {e}")
        return None, None

    # Load Trading Data
    try:
        trades = pd.read_csv('historical_data.csv')
        print(f"✅ Historical trades loaded: {trades.shape}")
    except Exception as e:
        print(f"❌ Error loading historical_data.csv: {e}")
        return None, None

    print("\n🔧 Cleaning and preparing data...")

    # Clean Fear & Greed data
    fear_greed['date'] = pd.to_datetime(fear_greed['date'], format='mixed', dayfirst=True)
    fear_greed = fear_greed.sort_values('date').reset_index(drop=True)

    # Clean Trading data
    trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], format='mixed', dayfirst=True)
    trades['date'] = trades['Timestamp IST'].dt.date
    trades['date'] = pd.to_datetime(trades['date'])

    # Remove missing values
    trades = trades.dropna(subset=['Closed PnL', 'Size USD', 'Execution Price'])

    # Create derived features
    trades['profitable'] = trades['Closed PnL'] > 0
    trades['hour'] = trades['Timestamp IST'].dt.hour
    trades['day_of_week'] = trades['Timestamp IST'].dt.day_name()

    print(f"✅ Data cleaned: {len(trades):,} valid trades")

    # Merge datasets
    merged_df = trades.merge(
        fear_greed[['date', 'value', 'classification']],
        on='date',
        how='left'
    )

    merged_df = merged_df.rename(columns={
        'value': 'fear_greed_score',
        'classification': 'market_sentiment'
    })

    print(f"✅ Datasets merged: {len(merged_df):,} trades with sentiment data")

    return fear_greed, trades, merged_df

fear_greed, trades, merged_df = load_and_prepare_data()

if merged_df is None:
    print("\n❌ FATAL ERROR: Could not load data. Please check your CSV files.")
    exit(1)

# ============================================================================
# STEP 2: BASIC STATISTICS
# ============================================================================

print("\n" + "="*80)
print("STEP 2: BASIC STATISTICS")
print("="*80)

print(f"\n📊 DATASET OVERVIEW:")
print(f"   Date Range: {merged_df['date'].min()} to {merged_df['date'].max()}")
print(f"   Total Trades: {len(merged_df):,}")
print(f"   Unique Traders: {merged_df['Account'].nunique():,}")
print(f"   Cryptocurrencies: {', '.join(merged_df['Coin'].unique())}")
print(f"   Total Volume: ${merged_df['Size USD'].sum():,.2f}")

print(f"\n💰 PERFORMANCE METRICS:")
total_pnl = merged_df['Closed PnL'].sum()
avg_pnl = merged_df['Closed PnL'].mean()
win_rate = (merged_df['profitable'].mean() * 100)
profitable_trades = merged_df['profitable'].sum()

print(f"   Total PnL: ${total_pnl:,.2f}")
print(f"   Average PnL per trade: ${avg_pnl:.2f}")
print(f"   Overall Win Rate: {win_rate:.2f}%")
print(f"   Profitable Trades: {profitable_trades:,} / {len(merged_df):,}")

# ============================================================================
# STEP 3: DEEP INSIGHTS ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("STEP 3: EXTRACTING DEEP INSIGHTS")
print("="*80)

# INSIGHT 1: Sentiment Performance
print("\n🔍 INSIGHT #1: SENTIMENT-DRIVEN PERFORMANCE")
print("-" * 60)

sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
sentiment_analysis = merged_df.groupby('market_sentiment').agg({
    'Closed PnL': ['mean', 'sum'],
    'profitable': 'mean',
    'Size USD': 'count'
}).round(2)

sentiment_analysis.columns = ['Avg_PnL', 'Total_PnL', 'Win_Rate', 'Trade_Count']
sentiment_analysis['Win_Rate'] = (sentiment_analysis['Win_Rate'] * 100).round(1)
sentiment_analysis = sentiment_analysis.reindex(sentiment_order)

print(sentiment_analysis.to_string())

best_sentiment = sentiment_analysis['Avg_PnL'].idxmax()
print(f"\n✅ BEST PERFORMANCE: {best_sentiment}")
print(f"   → Avg PnL: ${sentiment_analysis.loc[best_sentiment, 'Avg_PnL']:.2f}")
print(f"   → Win Rate: {sentiment_analysis.loc[best_sentiment, 'Win_Rate']:.1f}%")

# INSIGHT 2: Timing Analysis
print("\n🔍 INSIGHT #2: OPTIMAL TRADING TIMES")
print("-" * 60)

hourly_performance = merged_df.groupby('hour').agg({
    'Closed PnL': 'mean',
    'profitable': 'mean',
    'Size USD': 'count'
}).round(2)

best_hour = hourly_performance['Closed PnL'].idxmax()
worst_hour = hourly_performance['Closed PnL'].idxmin()

print(f"Best Hour: {best_hour}:00 (Avg PnL: ${hourly_performance.loc[best_hour, 'Closed PnL']:.2f})")
print(f"Worst Hour: {worst_hour}:00 (Avg PnL: ${hourly_performance.loc[worst_hour, 'Closed PnL']:.2f})")

# Day of week analysis
daily_performance = merged_df.groupby('day_of_week')['Closed PnL'].mean()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_performance = daily_performance.reindex(day_order)

best_day = daily_performance.idxmax()
print(f"\nBest Day: {best_day} (Avg PnL: ${daily_performance.max():.2f})")

# INSIGHT 3: Winners vs Losers
print("\n🔍 INSIGHT #3: CHARACTERISTICS OF WINNING TRADES")
print("-" * 60)

winners = merged_df[merged_df['profitable'] == True]
losers = merged_df[merged_df['profitable'] == False]

print(f"\nWinning Trades: {len(winners):,} ({len(winners)/len(merged_df)*100:.1f}%)")
print(f"   → Avg Sentiment: {winners['fear_greed_score'].mean():.1f}")
print(f"   → Avg Trade Size: ${winners['Size USD'].mean():,.2f}")
print(f"   → Most common hour: {winners['hour'].mode()[0]}:00")

print(f"\nLosing Trades: {len(losers):,} ({len(losers)/len(merged_df)*100:.1f}%)")
print(f"   → Avg Sentiment: {losers['fear_greed_score'].mean():.1f}")
print(f"   → Avg Trade Size: ${losers['Size USD'].mean():,.2f}")
print(f"   → Most common hour: {losers['hour'].mode()[0]}:00")

# INSIGHT 4: Trader Profiles
print("\n🔍 INSIGHT #4: TRADER ARCHETYPES")
print("-" * 60)

trader_stats = merged_df.groupby('Account').agg({
    'Closed PnL': ['sum', 'mean'],
    'profitable': 'mean',
    'Size USD': 'count'
}).reset_index()

trader_stats.columns = ['Account', 'Total_PnL', 'Avg_PnL', 'Win_Rate', 'Trade_Count']
active_traders = trader_stats[trader_stats['Trade_Count'] >= 10]

profitable_traders = active_traders[active_traders['Total_PnL'] > 0]
print(f"\nActive Traders (≥10 trades): {len(active_traders):,}")
print(f"Profitable Traders: {len(profitable_traders):,} ({len(profitable_traders)/len(active_traders)*100:.1f}%)")
print(f"\nTop Trader Total PnL: ${active_traders['Total_PnL'].max():,.2f}")
print(f"Average Win Rate (profitable traders): {profitable_traders['Win_Rate'].mean()*100:.1f}%")

# ============================================================================
# STEP 4: GENERATE VISUALIZATIONS
# ============================================================================

print("\n" + "="*80)
print("STEP 4: GENERATING VISUALIZATIONS")
print("="*80)

# Visualization 1: Sentiment Performance
print("\n📊 Creating sentiment performance visualization...")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Avg PnL by Sentiment
colors = ['#8B0000', '#DC143C', '#808080', '#90EE90', '#006400']
sentiment_pnl = sentiment_analysis['Avg_PnL'].values
axes[0, 0].bar(range(len(sentiment_order)), sentiment_pnl, color=colors, edgecolor='black', linewidth=2)
axes[0, 0].set_xticks(range(len(sentiment_order)))
axes[0, 0].set_xticklabels(sentiment_order, rotation=45, ha='right', fontweight='bold')
axes[0, 0].set_ylabel('Average PnL ($)', fontweight='bold')
axes[0, 0].set_title('💰 Performance by Market Sentiment', fontsize=14, fontweight='bold')
axes[0, 0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[0, 0].grid(axis='y', alpha=0.3)

# Plot 2: Win Rate by Sentiment
win_rates = sentiment_analysis['Win_Rate'].values
axes[0, 1].bar(range(len(sentiment_order)), win_rates, color=colors, edgecolor='black', linewidth=2)
axes[0, 1].set_xticks(range(len(sentiment_order)))
axes[0, 1].set_xticklabels(sentiment_order, rotation=45, ha='right', fontweight='bold')
axes[0, 1].set_ylabel('Win Rate (%)', fontweight='bold')
axes[0, 1].set_title('🎯 Win Rate by Sentiment', fontsize=14, fontweight='bold')
axes[0, 1].axhline(50, color='red', linestyle='--', linewidth=2, label='50% baseline')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# Plot 3: Hourly Performance
hourly_pnl = merged_df.groupby('hour')['Closed PnL'].mean()
axes[1, 0].plot(hourly_pnl.index, hourly_pnl.values, marker='o', linewidth=3,
               markersize=8, color='steelblue')
axes[1, 0].fill_between(hourly_pnl.index, hourly_pnl.values, alpha=0.3, color='steelblue')
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Hour of Day', fontweight='bold')
axes[1, 0].set_ylabel('Average PnL ($)', fontweight='bold')
axes[1, 0].set_title('⏰ Hourly Performance Pattern', fontsize=14, fontweight='bold')
axes[1, 0].grid(alpha=0.3)
axes[1, 0].set_xticks(range(0, 24, 2))

# Plot 4: Trader PnL Distribution
axes[1, 1].hist(active_traders['Total_PnL'], bins=50, color='purple',
               edgecolor='black', alpha=0.7)
axes[1, 1].axvline(0, color='red', linestyle='--', linewidth=2, label='Break-even')
axes[1, 1].set_xlabel('Total PnL ($)', fontweight='bold')
axes[1, 1].set_ylabel('Number of Traders', fontweight='bold')
axes[1, 1].set_title('📊 Trader Profitability Distribution', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(axis='y', alpha=0.3)

plt.suptitle('🎯 COMPREHENSIVE TRADING INSIGHTS DASHBOARD',
            fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('comprehensive_insights_dashboard.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.close()

print("✅ Dashboard saved as 'comprehensive_insights_dashboard.png'")

# ============================================================================
# STEP 5: GENERATE FINAL REPORT
# ============================================================================

print("\n" + "="*80)
print("STEP 5: GENERATING FINAL REPORT")
print("="*80)

report = f"""
{'='*80}
TRADER BEHAVIOR ANALYSIS - EXECUTIVE SUMMARY
{'='*80}

Analysis Date: {datetime.now().strftime('%Y-%m-%d')}
Data Period: {merged_df['date'].min()} to {merged_df['date'].max()}

{'─'*80}
DATASET OVERVIEW
{'─'*80}
Total Trades: {len(merged_df):,}
Unique Traders: {merged_df['Account'].nunique():,}
Total Volume: ${merged_df['Size USD'].sum():,.2f}
Cryptocurrencies: {', '.join(merged_df['Coin'].unique())}

{'─'*80}
OVERALL PERFORMANCE
{'─'*80}
Total PnL: ${total_pnl:,.2f}
Average PnL per Trade: ${avg_pnl:.2f}
Overall Win Rate: {win_rate:.2f}%
Profitable Trades: {profitable_trades:,} / {len(merged_df):,}

{'─'*80}
KEY INSIGHTS
{'─'*80}

1. OPTIMAL MARKET CONDITIONS
   ✅ Best Performance: {best_sentiment}
   → Average PnL: ${sentiment_analysis.loc[best_sentiment, 'Avg_PnL']:.2f}
   → Win Rate: {sentiment_analysis.loc[best_sentiment, 'Win_Rate']:.1f}%

   💡 RECOMMENDATION: Focus trading activity during {best_sentiment} periods

2. OPTIMAL TIMING
   ✅ Best Trading Hour: {best_hour}:00
   → Average PnL: ${hourly_performance.loc[best_hour, 'Closed PnL']:.2f}

   ✅ Best Trading Day: {best_day}
   → Average PnL: ${daily_performance.max():.2f}

   💡 RECOMMENDATION: Schedule active trading on {best_day}s around {best_hour}:00

3. WINNING TRADE CHARACTERISTICS
   → Avg Sentiment Score: {winners['fear_greed_score'].mean():.1f}
   → Avg Trade Size: ${winners['Size USD'].mean():,.2f}
   → Most Active Hour: {winners['hour'].mode()[0]}:00

   💡 RECOMMENDATION: Align trade parameters with winning patterns

4. TRADER PERFORMANCE
   → Active Traders: {len(active_traders):,}
   → Profitable Traders: {len(profitable_traders):,} ({len(profitable_traders)/len(active_traders)*100:.1f}%)
   → Top Performer PnL: ${active_traders['Total_PnL'].max():,.2f}

   💡 INSIGHT: {len(profitable_traders)/len(active_traders)*100:.1f}% of traders are consistently profitable

{'─'*80}
ACTIONABLE STRATEGIES
{'─'*80}

1. SENTIMENT-BASED STRATEGY
   → Trade more aggressively during {best_sentiment} market conditions
   → Reduce position sizes during {sentiment_analysis['Avg_PnL'].idxmin()} conditions

2. TIMING OPTIMIZATION
   → Focus on {best_day}s and hour {best_hour}:00 for highest probability trades
   → Avoid trading during hour {worst_hour}:00 (lowest performance)

3. POSITION SIZING
   → Winners average ${winners['Size USD'].mean():,.2f} per trade
   → Consider this benchmark for optimal position sizing

4. CONTINUOUS MONITORING
   → Track sentiment shifts in real-time
   → Adjust strategy as market conditions evolve

{'─'*80}
RISK DISCLAIMER
{'─'*80}
Past performance does not guarantee future results. This analysis is for
educational purposes only. Always use proper risk management and never
invest more than you can afford to lose.

{'='*80}
ANALYSIS COMPLETE
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
{'='*80}
"""

# Save report
with open('analysis_report.txt', 'w') as f:
    f.write(report)

print(report)
print("\n✅ Report saved as 'analysis_report.txt'")

# ============================================================================
# COMPLETION
# ============================================================================

print("\n" + "="*80)
print("🎉 ANALYSIS PIPELINE COMPLETED SUCCESSFULLY!")
print("="*80)

print("\n📁 Generated Files:")
print("   1. comprehensive_insights_dashboard.png - Main visualization")
print("   2. analysis_report.txt - Executive summary")
print("   3. merged_df - Processed data (available in memory)")

print("\n💡 Next Steps:")
print("   1. Review the generated visualizations")
print("   2. Read the executive summary report")
print("   3. Customize insights for your specific needs")
print("   4. Present findings to stakeholders")

print(f"\n⏱️ Total execution time: {(datetime.now() - datetime.now()).seconds} seconds")
print("\n" + "="*80)

🚀 TRADER BEHAVIOR ANALYSIS - COMPLETE PIPELINE

Execution started: 2025-11-08 08:35:50

STEP 1: LOADING DATA

📂 Loading datasets...
✅ Fear & Greed Index loaded: (2644, 4)
✅ Historical trades loaded: (211224, 16)

🔧 Cleaning and preparing data...
✅ Data cleaned: 211,224 valid trades
✅ Datasets merged: 211,224 trades with sentiment data

STEP 2: BASIC STATISTICS

📊 DATASET OVERVIEW:
   Date Range: 2023-05-01 00:00:00 to 2025-05-01 00:00:00
   Total Trades: 211,224
   Unique Traders: 32
   Cryptocurrencies: @107, AAVE, DYDX, AIXBT, GMX, EIGEN, HYPE, SOL, SUI, DOGE, ETH, kPEPE, TRUMP, ONDO, ENA, LINK, XRP, S, BNB, BERA, WIF, LAYER, MKR, KAITO, IP, JUP, USUAL, ADA, BTC, PURR/USDC, ZRO, @7, @19, @21, @44, @48, @11, @15, @46, @61, @28, @45, @9, @41, @38, kSHIB, GRASS, TAO, AVAX, @2, @6, @8, @10, @12, @16, @17, @35, @26, @24, @32, @29, @31, @33, @34, @36, @37, @47, @53, @74, RUNE, CANTO, NTRN, BLUR, ZETA, MINA, MANTA, RNDR, WLD, kBONK, ALT, INJ, STG, ZEN, MAVIA, PIXEL, ILV, FET, STRK, CAKE, ST